<a href="https://colab.research.google.com/github/masadream/data_analytics/blob/master/Causal_Inference_studysession/Ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.4.1

##### モジュール準備


In [0]:
import pandas as pd
import numpy as np
from statsmodels.stats.weightstats import ttest_ind

##### データセットダウンロード

In [0]:
email_data = pd.read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

##### 女性向けメール配信データをデータセットから除外

In [0]:
male_df = email_data[email_data["segment"] != "Womens E-Mail"]

##### 介入を表すtreatment変数を追加

In [4]:
male_df["treatment"] = np.where(male_df["segment"] == "Mens E-Mail", 1, 0)

NameError: ignored